In [1]:
from notebook_utils import add_project_path
add_project_path()
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import datetime
import features as f 
from match_station import coord_merge, stations_list, coord_km
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 200)

In [2]:
geo_data = pd.read_csv('../data_raw/geo_data.csv')
atmos = pd.read_csv('../data_raw/atmos.csv', index_col=0)
meteo = pd.read_csv('../data_raw/meteo.csv', index_col=0)
repair = pd.read_csv('../data_raw/repair.csv', parse_dates=['datetime']) 
crash = pd.read_csv('../data_raw/crash_parts.csv', parse_dates=['datetime'], index_col=0)
tele2 = pd.read_csv('../data_raw/tele2_data.csv', parse_dates=['datetime'])
traffic = pd.read_csv('../data_raw/traffic.csv', parse_dates=['datetime'])
train = pd.read_csv('../data_raw/train.csv', parse_dates=['datetime'])
test = pd.read_csv('../data_raw/test.csv', parse_dates=['datetime'])

/Users/evgenyesingildinov/.local/share/virtualenvs/mchs-is38nFyi/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
traffic_p = f.preprocess_traffic(traffic)
geo_data_p = f.preprocess_geo(geo_data)

In [4]:
traffic_stations = traffic[['station_id', 'road_id', 'road_km', 'latitude', 'longitude']].drop_duplicates().sort_values(['road_id', 'road_km']).reset_index(drop=True)

In [5]:
geo_data_p = geo_data_p.merge(traffic_stations[['station_id', 'road_id', 'road_km']], on=['road_id', 'road_km'], how='left')

In [6]:
geo_data_p.loc[geo_data_p.road_id == 9, 'station_id_match'] = geo_data_p.loc[geo_data_p.road_id == 9, 'station_id'].interpolate(method='nearest').ffill().bfill()
geo_data_p.loc[geo_data_p.road_id == 14, 'station_id_match'] = geo_data_p.loc[geo_data_p.road_id == 14, 'station_id'].interpolate(method='nearest').ffill().bfill()

In [7]:
geo_data_p = geo_data_p.merge(traffic_stations[['station_id', 'latitude', 'longitude']], 
                              left_on='station_id_match', right_on='station_id',
                              how='left')

geo_data_p['dist_nearest_station'] = np.sqrt((geo_data_p['latitude'] - geo_data_p['lat_geoc'])**2 +
                                             (geo_data_p['longitude'] - geo_data_p['lon_geoc'])**2)

geo_data_p.drop(columns=['km_name', 'station_id_x', 'station_id_y', 'latitude', 'longitude'], inplace=True)
geo_data_p.rename(columns={'station_id_match': 'station_id'}, inplace=True)


In [8]:
meteo_p = f.preprocess_meteo(meteo)
meteo_p = coord_merge(meteo_p)
geo_data_p = coord_km(geo_data_p)

In [9]:
geo_data_p['lat'], geo_data_p['lon'] = geo_data_p['lat_long'].str
geo_data_p = geo_data_p[~(geo_data_p.lat.isnull() | geo_data_p.lon.isnull())].reset_index(drop=True)

<ipython-input-9-e2e63980c1fe>:1: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  geo_data_p['lat'], geo_data_p['lon'] = geo_data_p['lat_long'].str


In [10]:
geo_data_p = f.add_meteo_stations(geo_data_p, meteo_p)

In [11]:
geo_data_p.drop(columns=['lat_long', 'Широта', 'Долгота'], inplace=True)

In [12]:
# train_p = f.preprocess_train(train)
# train_x = f.compile_train(train_p,
#                           start_date='2016-01-01 00:00:00',
#                           end_date='2021-01-01 00:00:00')
# train_x.to_csv('../data_processed/train.csv.gz')
train_x = pd.read_csv('../data_processed/train.csv.gz', index_col=0, parse_dates=['datetime'])

/Users/evgenyesingildinov/.local/share/virtualenvs/mchs-is38nFyi/lib/python3.9/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [13]:
train_x.head()

,datetime,segment_id,target,road_id,road_km
0,2016-01-01,14_1276,0,14,1276
1,2016-01-01,9_278,0,9,278
2,2016-01-01,14_432,0,14,432
3,2016-01-01,14_1,0,14,1
4,2016-01-01,9_451,0,9,451


In [14]:
train_x = pd.merge(train_x, geo_data_p, on=['road_id', 'road_km'], how='left')

In [15]:
train_x = pd.merge(train_x, traffic_p, on=['datetime', 'station_id'], how='left')

In [22]:
train_x.drop(columns=['lat_glonass', 'lon_glonass'], inplace=True)

In [ ]:
train_x.to_pickle('../data_processed/train_traffic.pickle.gz')

In [ ]:
train_x = pd.merge(train_x, tele2, on=['datetime', 'road_id', 'road_km'], how='left')

In [42]:
# train_x = pd.merge(train_p, traffic_p, how='outer', on=['datetime', 'road_id', 'road_km'])
# condition1 = (train_x['target'].isnull()) & (train_x['data_id'].isnull())
# condition2 = ~(train_x['target'].isnull()) & (train_x['data_id'].isnull())
# train_x = train_x.drop(train_x[condition1 | condition2].index)

# train_x['target'].fillna(0, inplace=True)
# train_x.sort_values('datetime', inplace=True)
# train_x.reset_index(drop=True, inplace=True)

# train_x = train_x[(train_x.datetime >= '2016-01-01 00:00:00') & (train_x.datetime <= '2020-01-01 00:00:00')]
# train_x = f.add_segment_id(train_x)